In [1]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# pip3 install lxml

# load secrets
load_dotenv()

True

In [2]:
def get_historical_data(symbol, lookback_days=90):
    iex_key = os.getenv("IEX_API_KEY") 
    historical_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/chart/{lookback_days}d?token={iex_key}'

    historical_data = requests.get(historical_url).json()
    df = pd.DataFrame(historical_data)
    return df[['date', 'high', 'low', 'close']]

# Fetch historical data
ticker = 'PLTR' # Palantir Technologies Inc
#### sudz4 -> think about getting a list, can do list of SP500 or sector, pass the list to the function and get the data
historical_df = get_historical_data(ticker)

# Display the historical data
# print(historical_df)

# Convert the 'date' column to datetime objects
historical_df['date'] = pd.to_datetime(historical_df['date'])

# Sort the DataFrame by the 'date' column
historical_df = historical_df.sort_values(by='date')

In [11]:
#### DEV -> df view options
# print(historical_df) # full view

print('Historical Data START')
print(historical_df.head(5))
print()
print('Historical Data END')
print(historical_df.tail(5)) # last 5 rows

# print(historical_df.head(1)) # first row
# print(historical_df.tail(1)) # last row

Historical Data START
        date   high    low  close
0 2023-11-06  19.08  17.93  18.54
1 2023-11-07  19.17  18.53  18.80
2 2023-11-08  18.93  18.15  18.49
3 2023-11-09  18.87  18.16  18.27
4 2023-11-10  19.72  18.06  19.67

Historical Data END
         date     high    low  close
55 2024-01-26  16.7500  16.34  16.35
56 2024-01-29  17.1100  16.43  17.09
57 2024-01-30  17.2400  16.66  16.74
58 2024-01-31  16.7600  16.07  16.09
59 2024-02-01  16.4868  16.03  16.33
